# Protección de datos personales

Es necesario proteger los datos de la compañía de seguros "Al menos inundar". Desarrollar un método de conversión de datos para dificultar la recuperación de información personal. Explicar la corrección de su funcionamiento.

Es necesario proteger los datos para que la calidad de los modelos de aprendizaje automático no se deteriore durante la transformación. No es necesario seleccionar el mejor modelo.

#### Plan de Acción

1. Descargar y revisar los datos.
2. Responda a la pregunta y explique la solución. 
 Las características se multiplican por la matriz invertible. ¿Cambiará la calidad de la regresión lineal? (Se puede enseñar de nuevo.)  
 a. Cambiará. Proporcione ejemplos de matrices.  
 b. No cambiará. Especifique cómo se relacionan los parámetros de regresión lineal en el original y el convertido.
3. Proponer un algoritmo de conversión de datos para resolver el problema. Explicar por qué la calidad de la regresión lineal no cambiará.
4. Programe este algoritmo usando operaciones de matriz. Compruebe que la calidad de la regresión lineal de sklearn es la misma antes y después de la conversión. Aplique la métrica R2.

## Carga de datos

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Ni permisos.

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


En las columnas Edad y Salario puede cambiar el conjunto para ahorrar recursos de procesamiento.

In [4]:
df[['Зарплата', 'Возраст']] = df[['Зарплата', 'Возраст']].astype('int')


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


Buscaremos duplicados por si acaso.

In [6]:
df.duplicated().sum()

153

In [7]:
df = df.drop_duplicates()

Se han eliminado los duplicados. Puede continuar con el estudio.

## Multiplicación de matrices

Designación:

- $X$ - matriz de características (la columna cero consta de unidades)

- $y$ - vector de función objetivo

- $P$ - matriz multiplicada por características

- $w$ - vector de pesos de regresión lineal (elemento cero es desplazamiento)

Predicciones:

$$
a = Xw
$$

Objetivo de la formación:

$$
w = argmin_w MSE(Xw, y)
$$

Fórmula de aprendizaje:

$$
w = (X T X) {-1} X T y
$$

Las características se multiplican por la matriz invertible - ¿cambiará la calidad de la regresión lineal?

**Respuesta:** La calidad de la regresión lineal no cambiará.

**Justificación:**

Predicción para nueva matriz: $ a_P = XPw $

Fórmula del vector de peso: $ w = ((XP) T (XP)) {-1} (XP) T y $


Sustituya el valor del vector de pesos:
 
$$ a_P = (XP)((XP) T (XP)) {-1} (XP) T y $$

Abrir los paréntesis según la propiedad: $ a_P = (XP)(P TX TXP) {-1}P TX T y $

Quite P de los corchetes: $ a_P = XPP {-1}(X TX) {-1}(P T) {-1}P TX Ty $

Porque cuando se multiplica por la matriz inversa, obtenemos una unidad: $ (P T) {-1}  P T = E $ $,

Obtener: $ a_P = X(X T X) {-1}X T Y $

Por lo tanto: $ a_P= Xw=a $

La fórmula original del vector de predicción era igual a la fórmula del vector de predicción en el caso de que la matriz de características se multiplicara por la matriz invertible. La calidad de la regresión lineal no ha cambiado.

## Algoritmo de transformación

***Algoritmo*

1. Generar ancho de matriz cuadrado invertible igual al número de características (4x4)
2. Multiplicar la matriz de características original por la generada
3. Calcular la calidad del modelo en la matriz resultante y compararla con la calidad del modelo en la matriz de datos de entrada.

**Justificación**

Encontramos que las predicciones de la matriz modificada y original no deben diferir, por lo que los resultados de la métrica R2 deben ser iguales. De esta manera podemos demostrar que hemos protegido correctamente nuestros datos.

## Comprobación de algoritmos

In [8]:
# característica original

features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

In [9]:
# generar matriz invertible

matrix_generated = np.random.normal(size=(4, 4))
display (matrix_generated)

array([[-1.2863119 ,  1.67646465,  0.18809933,  0.33235236],
       [ 1.48453311, -0.42958879, -0.11353671, -0.53417479],
       [-1.11981203,  0.31471632,  0.27397941,  0.44891385],
       [ 0.14097508,  0.96625637, -1.70894976, -1.32300398]])

In [10]:
# hacer matriz inversa (en caso de que el original sea irreversible - será un error)

matrix_invert = np.linalg.inv(matrix_generated)
display (matrix_invert)

array([[  0.57678892,  -2.08232863,  -4.37907034,  -0.5002258 ],
       [  0.87359156,  -0.59794716,  -1.81572027,  -0.15521757],
       [ -0.18618215,   6.04129523,   8.26279452,   0.31768403],
       [  0.93998373,  -8.46225479, -12.4659427 ,  -1.33287988]])

In [11]:
# crear una nueva matriz, multiplicando la matriz de características original por la generada

matrix = features.values
matrix_new = matrix @ matrix_invert
features_incode = pd.DataFrame (matrix_new, columns = features.columns)

In [12]:
features_incode

,Пол,Возраст,Зарплата,Члены семьи
0,-9197.300769,299613.183183,409743.318514,15748.930939
1,-7033.796623,229533.251063,313890.202582,12063.520311
2,-3884.491060,126849.859444,173466.028975,6666.863353
3,-7743.570395,251892.529853,344495.469360,13241.498787
4,-4834.316844,157658.980755,215603.717663,8286.706906
...,...,...,...,...
4842,-6620.362336,215640.572819,294905.992214,11334.308074
4843,-9725.302727,316535.077797,432896.232274,16640.032976
4844,-6292.223192,204771.024976,280047.487845,10763.718558
4845,-6065.540653,197509.730217,270111.657971,10380.354180


In [13]:
# crear una clase para enseñar modelos

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T@X)@X.T@y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [14]:
# Vamos a entrenar el modelo en las características originales y deducir r2

model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4302010046633359


In [15]:
# enseñar el modelo en signos codificados e imprimir r2
model = LinearRegression()
model.fit(features_incode, target)
predictions = model.predict(features_incode)
print(r2_score(target, predictions))

0.4302010046514576


r2 métricas son casi las mismas - protección de datos es exitoso!